#OCR for school profile

In [1]:
!git clone https://github.com/ultralytics/yolov5.git
!pip install -r yolov5/requirements.txt
#!pip install torchvision==0.13
!pip install wandb

Cloning into 'yolov5'...
remote: Enumerating objects: 16634, done.
remote: Counting objects: 100% (175/175), done.
remote: Compressing objects: 100% (128/128), done.
remote: Total 16634 (delta 70), reused 106 (delta 47), pack-reused 16459
Receiving objects: 100% (16634/16634), 15.34 MiB | 32.39 MiB/s, done.
Resolving deltas: 100% (11369/11369), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 726.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 782.8/782.8 kB 39.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.5.0
    Uninstalling Pillow-9.5.0:
      Successfully uninstalled Pillow-9.5.0
ERROR: pip's de

In [2]:
import os
import shutil
import wandb
wandb.login(key='e9f98fa97698b1759125dc076e2195f407c2d4af')

gd_label_dir = '/kaggle/input/school-profile/HB/Labeled'
label_files = os.listdir(gd_label_dir)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
gd_img_dir = '/kaggle/input/school-profile/HB/Original'
dataset_folder = '/kaggle/working/yolov5/dataset/custom_data'
os.makedirs(dataset_folder, exist_ok=True)

img_folders = os.listdir(gd_img_dir)
#os.makedirs(img_dir, exist_ok=True)
#print(*label_files, sep='\n')

file_names = set([label_file.split('/')[-1][:-7] for label_file in label_files])
for name in file_names:
    if name in img_folders:
        imgs = os.listdir(os.path.join(gd_img_dir, name))
        #print(imgs)
        for img in imgs:
            shutil.copy(os.path.join(gd_img_dir, name, img), dataset_folder)
            shutil.copy(os.path.join(gd_label_dir, img[:-3]+'txt'), dataset_folder)

In [4]:
import random

valid_test_size = 0.3

valid_samples = set(random.sample(img_folders, int(valid_test_size*len(img_folders))))
train_samples = set(img_folders) - valid_samples
test_samples = set(random.sample(valid_samples, int(0.25*len(valid_samples))))
valid_samples = valid_samples - test_samples

print("test_samples:", len(test_samples))
print("train_samples:", len(train_samples))
print("valid_samples:", len(valid_samples))

with open("/kaggle/working/yolov5/dataset/custom_train.txt", 'w') as f:
    for sample in train_samples:
        imgs = os.listdir(os.path.join(gd_img_dir, sample))
        for img in imgs:
            f.write('dataset/custom_data/'+img+'\n')

with open("/kaggle/working/yolov5/dataset/custom_valid.txt", 'w') as f:
    for sample in valid_samples:
        imgs = os.listdir(os.path.join(gd_img_dir, sample))
        for img in imgs:
            f.write('dataset/custom_data/'+img+'\n')

with open("/kaggle/working/yolov5/dataset/custom_test.txt", 'w') as f:
    for sample in test_samples:
        imgs = os.listdir(os.path.join(gd_img_dir, sample))
        for img in imgs:
            f.write('dataset/custom_data/'+img+'\n')

test_samples: 4
train_samples: 39
valid_samples: 12


/tmp/ipykernel_25/3331717593.py:7: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  test_samples = set(random.sample(valid_samples, int(0.25*len(valid_samples))))


In [5]:
classes_file = '/kaggle/input/school-profile/HB/Labeled/classes.txt'
with open(classes_file, 'r') as f:
    classes = f.read().strip().split('\n')
custom_content = f"path: ./dataset\n\
train: custom_train.txt\n\
val:  custom_valid.txt\n\
test:  custom_train_test.txt\n\
nc: {len(classes)}  # number of classes\n\
names: {classes}  # class names\n\
"

with open('/kaggle/working/yolov5/data/custom.yaml', 'w') as f:
    f.write(custom_content)

In [6]:
os.chdir('/kaggle/working/yolov5/')
os.getcwd()

'/kaggle/working/yolov5'

In [7]:
#!python /content/Easy-Yolo-OCR/main.py --gpu 0 --lang en
!python train.py --data data/custom.yaml --weights yolov5s.pt --img 640 --batch-size 32 --epochs 150

wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2024-06-18 03:24:17.244970: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-18 03:24:17.245073: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-18 03:24:17.354323: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: Currently logged in as: thanh_nd (thanh_nd_ai). Use `wandb login --relogin` to force relogin
train: weights=yolov5s.pt, cfg=, data=data/custom.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=150, 